In [1]:
import cPickle as pickle
import gzip
import numpy as np
import nnetwork as nn

The mnist data set can, e.g., be found on this webpage:
http://deeplearning.net/tutorial/gettingstarted.html
Here, we just store the datafile in the dataset directory.

In [2]:
fp = './datasets/mnist.pkl.gz'
with gzip.open(fp, 'rb') as f:
    train_set, valid_set, test_set = pickle.load(f)

We need to transform the target values into one hot encoding.
For this we define a small helper function.

In [3]:
def one_hot_encoding(targets):
    n_examples = targets.shape[0]
    one_hot_targets = np.zeros((n_examples, 10))
    idx = np.arange(n_examples)
    one_hot_targets[idx, targets] = 1.
    return one_hot_targets

target_train = one_hot_encoding(train_set[1])
target_valid = one_hot_encoding(valid_set[1])
test_valid = one_hot_encoding(test_set[1])

We are going to implement a very basic loop for hyper parameter tuning of the l2-regularisation.
In each loop occurence, we create a neural network with a different l2 value and train until our train-criterion is broken. We stop training if the loss after one epoch is only marginally lower than the previous epoch's loss by the value of the precision variable, which we set in the fit function of the neural network. We then evaluate the accuracy on the validation set and chose the best l2 parameter based on this metric. (We could also use the loss on the validation set)

To initialise the network, we also have to define the number of layers and their number of nodes. Note, that only DenseLayers are implemented here thus far. We go for a very basic network to test our implementation. We connect the input layer directly with the output layer.

The parameters of the network are initialised randomly from a Gaussian distribution with mean = 0. and sigma = 0.5.

In [4]:
n_layers = np.array([784, 10])
l2s = [0.01, 0.1, 1.]
best_acc = 0.
best_net = None
for l2 in l2s:
    net = nn.NeuralNet(n_layers, l2)
    net.train(train_set[0].T, target_train.T, precision=0.005)
    y_pred = net.predict(valid_set[0].T)
    acc = 100 * np.mean(np.where(y_pred==valid_set[1], 1, 0))
    print 'final validation accuracy: {}'.format(acc)
    if acc >= best_acc:
        best_net = net
        best_acc = acc



NEURAL NETWORK created!
2 layers in total (including In- and Output-layer)
1 parameter matrices

0 1.2 17.77255542234628
10 1.2 2.355281528974366
Final Cost: 2.1825213880962973
final validation accuracy: 68.98

NEURAL NETWORK created!
2 layers in total (including In- and Output-layer)
1 parameter matrices

0 1.2 18.248673395708515
10 0.6 2.4111839301868603
20 0.6 1.9210821368936233
30 0.6 1.7003980494574713
40 0.6 1.5595286940342683
50 0.6 1.4600579776799822
60 0.6 1.385138351238439
70 0.6 1.3261126613191232
Final Cost: 1.30576873033237
final validation accuracy: 83.25

NEURAL NETWORK created!
2 layers in total (including In- and Output-layer)
1 parameter matrices

0 1.2 25.650289376899973
10 1.2 2.225695060282258
Final Cost: 1.8139319853500517
final validation accuracy: 75.68


Now that we trained the networks and found the best value of the l2 parameter, we would like to evaluate the final accuracies, for the train, validation and test set.
For this, we simply use the predict function of the neural network:

In [5]:
y_pred = best_net.predict(train_set[0].T)
print 'final train accuracy: {}'.format(100 * np.mean(np.where(y_pred==train_set[1], 1, 0)))

final train accuracy: 81.912


In [6]:
y_pred = best_net.predict(valid_set[0].T)
print 'final validation accuracy: {}'.format(100 * np.mean(np.where(y_pred==valid_set[1], 1, 0)))

final validation accuracy: 83.25


In [7]:
y_pred = best_net.predict(test_set[0].T)
print 'final test accuracy: {}'.format(100 * np.mean(np.where(y_pred==test_set[1], 1, 0)))

final test accuracy: 83.03


We see that we can get a decent accuracy of hand-written digits recognition with our implementation of neural networks. This confirms that our implementation is correct (including the backpropagation).

It is surprising, that simply by connecting the input to the output Layer, we can get an accuray on the test set of 83%.